<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 2
---------------------------------------------------------------
Student numbers: <span style="color:red">r1, r2, r3, r4, r5</span>. (fill in your student numbers!)

In this group assignment your team will delve into some deep learning applications for computer vision. The assignment will be delivered in the same groups from *Group assignment 1* and you start from this template notebook. The notebook you submit for grading is the last notebook you submit in the [Kaggle competition](https://www.kaggle.com/t/d11be6a431b84198bc85f54ae7e2563f) prior to the deadline on **Tuesday 24 May 23:59**. Closely follow [these instructions](https://github.com/gourie/kaggle_inclass) for joining the competition, sharing your notebook with the TAs and making a valid notebook submission to the competition. A notebook submission not only produces a *submission.csv* file that is used to calculate your competition score, it also runs the entire notebook and saves its output as if it were a report. This way it becomes an all-in-one-place document for the TAs to review. As such, please make sure that your final submission notebook is self-contained and fully documented (e.g. provide strong arguments for the design choices that you make). Most likely, this notebook format is not appropriate to run all your experiments at submission time (e.g. the training of CNNs is a memory hungry and time consuming process; due to limited Kaggle resources). It can be a good idea to distribute your code otherwise and only summarize your findings, together with your final predictions, in the submission notebook. For example, you can substitute experiments with some text and figures that you have produced "offline" (e.g. learning curves and results on your internal validation set or even the test set for different architectures, pre-processing pipelines, etc). We advise you to first go through the PDF of this assignment entirely before you really start. Then, it can be a good idea to go through this notebook and use it as your first notebook submission to the competition. You can make use of the *Group assignment 2* forum/discussion board on Toledo if you have any questions. Good luck and have fun!

---------------------------------------------------------------
NOTES:
* This notebook is just a template. Please keep the five main sections, but feel free to adjust further in any way you please!
* Clearly indicate the improvements that you make! You can for instance use subsections like: *3.1. Improvement: applying loss function f instead of g*.


# 1. Overview
This assignment consists of *three main parts* for which we expect you to provide code and extensive documentation in the notebook:
* Image classification (Sect. 2)
* Semantic segmentation (Sect. 3)
* Adversarial attacks (Sect. 4)

In the first part, you will train an end-to-end neural network for image classification. In the second part, you will do the same for semantic segmentation. For these two tasks we expect you to put a significant effort into optimizing performance and as such competing with fellow students via the Kaggle competition. In the third part, you will try to find and exploit the weaknesses of your classification and/or segmentation network. For the latter there is no competition format, but we do expect you to put significant effort in achieving good performance on the self-posed goal for that part. Finally, we ask you to reflect and produce an overall discussion with links to the lectures and "real world" computer vision (Sect. 5). It is important to note that only a small part of the grade will reflect the actual performance of your networks. However, we do expect all things to work! In general, we will evaluate the correctness of your approach and your understanding of what you have done that you demonstrate in the descriptions and discussions in the final notebook.

## 1.1 Deep learning resources
If you did not yet explore this in *Group assignment 1 (Sect. 2)*, we recommend using the TensorFlow and/or Keras library for building deep learning models. You can find a nice crash course [here](https://colab.research.google.com/drive/1UCJt8EYjlzCs1H1d1X0iDGYJsHKwu-NO).

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Input,BatchNormalization,Activation,Flatten, Dropout
from keras.models import Model
from keras.applications.vgg16 import VGG16
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow as tf
import keras
import cv2

print("Load modules complete!")

## 1.2 PASCAL VOC 2009
For this project you will be using the [PASCAL VOC 2009](http://host.robots.ox.ac.uk/pascal/VOC/voc2009/index.html) dataset. This dataset consists of colour images of various scenes with different object classes (e.g. animal: *bird, cat, ...*; vehicle: *aeroplane, bicycle, ...*), totalling 20 classes.

In [ ]:
# Loading the training data
train_df = pd.read_csv('/kaggle/input/kul-h02a5a-computer-vision-ga2-2023/train/train_set.csv', index_col="Id")
labels = train_df.columns
train_df["img"] = [np.load('/kaggle/input/kul-h02a5a-computer-vision-ga2-2023/train/img/train_{}.npy'.format(idx)) for idx, _ in train_df.iterrows()]
train_df["seg"] = [np.load('/kaggle/input/kul-h02a5a-computer-vision-ga2-2023/train/seg/train_{}.npy'.format(idx)) for idx, _ in train_df.iterrows()]
print("The training set contains {} examples.".format(len(train_df)))

# Show some examples
# fig, axs = plt.subplots(2, 20, figsize=(10 * 20, 10 * 2))
# for i, label in enumerate(labels):
#     df = train_df.loc[train_df[label] == 1]
#     axs[0, i].imshow(df.iloc[0]["img"], vmin=0, vmax=255)
#     axs[0, i].set_title("\n".join(label for label in labels if df.iloc[0][label] == 1), fontsize=40)
#     axs[0, i].axis("off")
#     axs[1, i].imshow(df.iloc[0]["seg"], vmin=0, vmax=20)  # with the absolute color scale it will be clear that the arrays in the "seg" column are label maps (labels in [0, 20])
#     axs[1, i].axis("off")
    
# plt.show()

# # The training dataframe contains for each image 20 columns with the ground truth classification labels and 20 column with the ground truth segmentation maps for each class
# train_df.head(1)

In [ ]:
def make_palette(num_classes):
    """
    Maps classes to colors in the style of PASCAL VOC.
    Close values are mapped to far colors for segmentation visualization.
    See http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html#devkit

    Takes:
        num_classes: the number of classes
    Gives:
        palette: the colormap as a k x 3 array of RGB colors
    """
    palette = np.zeros((num_classes, 3), dtype=np.uint8)
    for k in range(0, num_classes):
        label = k
        i = 0
        while label:
            palette[k, 0] |= (((label >> 0) & 1) << (7 - i))
            palette[k, 1] |= (((label >> 1) & 1) << (7 - i))
            palette[k, 2] |= (((label >> 2) & 1) << (7 - i))
            label >>= 3
            i += 1
    return palette

def color_seg(seg, palette):
    """
    Replace classes with their colors.

    Takes:
        seg: H x W segmentation image of class IDs
    Gives:
        H x W x 3 image of class colors
    """
    return palette[seg.flat].reshape(seg.shape + (3,))

In [ ]:
colormap = make_palette(21)
df_labels = [train_df.columns[:-2][i] for i in range(len(labels))]
classes = ['background'] + df_labels 
colormap2class = np.zeros(256 ** 3)      # Shape:(16777216,)
for i,cm in enumerate(colormap):
    colormap2class[(cm[0] * 256 + cm[1]) * 256 + cm[2]] = i # build index

"""
Key: index of color map i.e [128, 0, 0] = 1
Val: the array describing color map i.e [128, 0, 0]

"""
id_colormap = {k:v for k,v in enumerate(colormap)}

In [ ]:
plt.imshow(color_seg(train_df['seg'][0], palette))

In [ ]:
import sys
np.set_printoptions(threshold=sys.maxsize)
print(train_df["seg"][0].shape)
print(train_df["seg"][0][2].shape)
# print(train_df["seg"][0])


plt.imshow(train_df["seg"][0], vmin=0, vmax=3)
# train_df["seg"][0]

In [ ]:
# Loading the test data
test_df = pd.read_csv('/kaggle/input/kul-h02a5a-computer-vision-ga2-2023/test/test_set.csv', index_col="Id")
test_df["img"] = [np.load('/kaggle/input/kul-h02a5a-computer-vision-ga2-2023/test/img/test_{}.npy'.format(idx)) for idx, _ in test_df.iterrows()]
test_df["seg"] = [-1 * np.ones(img.shape[:2], dtype=np.int8) for img in test_df["img"]]
print("The test set contains {} examples.".format(len(test_df)))

# The test dataframe is similar to the training dataframe, but here the values are -1 --> your task is to fill in these as good as possible in Sect. 2 and Sect. 3; in Sect. 6 this dataframe is automatically transformed in the submission CSV!
test_df.head(1)

## 1.3 Your Kaggle submission
Your filled test dataframe (during Sect. 2 and Sect. 3) must be converted to a submission.csv with two rows per example (one for classification and one for segmentation) and with only a single prediction column (the multi-class/label predictions running length encoded). You don't need to edit this section. Just make sure to call this function at the right position in this notebook.

In [ ]:
def _rle_encode(img):
    """
    Kaggle requires RLE encoded predictions for computation of the Dice score (https://www.kaggle.com/lifa08/run-length-encode-and-decode)

    Parameters
    ----------
    img: np.ndarray - binary img array
    
    Returns
    -------
    rle: String - running length encoded version of img
    """
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    rle = ' '.join(str(x) for x in runs)
    return rle

def generate_submission(df):
    """
    Make sure to call this function once after you completed Sect. 2 and Sect. 3! It transforms and writes your test dataframe into a submission.csv file.
    
    Parameters
    ----------
    df: pd.DataFrame - filled dataframe that needs to be converted
    
    Returns
    -------
    submission_df: pd.DataFrame - df in submission format.
    """
    df_dict = {"Id": [], "Predicted": []}
    for idx, _ in df.iterrows():
        df_dict["Id"].append(f"{idx}_classification")
        df_dict["Predicted"].append(_rle_encode(np.array(df.loc[idx, labels])))
        df_dict["Id"].append(f"{idx}_segmentation")
        df_dict["Predicted"].append(_rle_encode(np.array([df.loc[idx, "seg"] == j + 1 for j in range(len(labels))])))
    
    submission_df = pd.DataFrame(data=df_dict, dtype=str).set_index("Id")
    submission_df.to_csv("submission.csv")
    return submission_df

# 3. Semantic segmentation
The goal here is to implement a segmentation CNN that labels every pixel in the image as belonging to one of the 20 classes (and/or background). Use the training set to train your CNN and compete on the test set (by filling in the segmentation column in the test dataframe).

In [ ]:
class RandomSegmentationModel:
    """
    Random segmentation model: 
        - generates random label maps for the inputs based on the class distributions observed during training
        - every pixel in an input can only have one label
    """
    def fit(self, X, Y):
        """
        Adjusts the class ratio variable to the one observed in Y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        Y: list of arrays - n x (height x width)

        Returns
        -------
        self
        """
        self.distribution = np.mean([[np.sum(Y_ == i) / Y_.size for i in range(len(labels) + 1)] for Y_ in Y], axis=0)
        print("Setting class distribution to:\nbackground: {}\n{}".format(self.distribution[0], "\n".join(f"{label}: {p}" for label, p in zip(labels, self.distribution[1:]))))
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label map.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        Y_pred: list of arrays - n x (height x width)
        """
        np.random.seed(0)
        return [np.random.choice(np.arange(len(labels) + 1), size=X_.shape[:2], p=self.distribution) for X_ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
model = RandomSegmentationModel()
model.fit(train_df["img"], train_df["seg"])
test_df.loc[:, "seg"] = model.predict(test_df["img"])
test_df.head(1)

## 3.2 Transfer Learning on Semantic Segmentation 
As seen in the performance of the trained model from scrath efficiency can be improved. For this matter, we will train a model using transfer learning.
In Semantic Segmentation several architectures are found, amount popular optiosn are  VGG-UNet, FastFCN, DeepLab among others. We will be using the first approach namely VGG16, for more details visit: https://keras.io/api/applications/#inception3

### 3.2.1 Analysig the data

In [ ]:
# The dimensions of each image are diff
print('Inspecting img and smg sizes:')
for i in range(4):
    print("Size img:",train_df["img"][i].shape)
    print("Size lab:", train_df["seg"][i].shape)

In [ ]:
# Show some examples of img and its corresponding smg
import warnings
import matplotlib
warnings.filterwarnings("ignore", category=matplotlib.MatplotlibDeprecationWarning)

def plot_img_seg(original_img, real_label, predicted_label=None):
    fig, ax = plt.subplots(figsize=(8, 8))
    col = 2
    if predicted_label is not None:
        col = 3
    # First Img
    plt.subplot(int(f"1{col}1"))
    plt.imshow(original_img)
    plt.title("Original Image")
    
    # Second Img
    plt.subplot(int(f"1{col}2"))
    plt.imshow(real_label)
    plt.title("Truth Label")
    
    # Third Img
    if predicted_label is not None:
        plt.subplot(int(f"1{col}3"))
        plt.imshow(predicted_label)
        plt.title("Predicted Label")
        

In [ ]:
import cv2
gray = train_df["seg"][0]
color = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)

def get_labelled_seg(img):
    """
     For segmentation task, read color image, transform it to 3-d one-hot encodeing class target image.
    :param img_path: the image path
    :return: target: (image_size,image_size,21)
    """
    image_size = 224
#     colored_img = cv2.cvtColor(img,cv2.COLOR_GRAY2RGB)
    colored_img = color_seg(img, palette)
    plt.imshow(colored_img)
    
    resized_img = resize(colored_img, (image_size, image_size))
    img_224 = np.array(resized_img).astype('int32')    
    print(img_224.shape)
    
    idx = (img_224[:, :, 0] * 256 + img_224[:, :, 1]) * 256 + img_224[:, :, 2]
    print('id shape',idx.shape)
    img_mapped = np.array(colormap2class[idx], dtype='int64') # 2-d class map, class of every pixel
    
    return img_mapped

a = get_labelled_seg(train_df['seg'][0])
# plt.imshow(a)
# plt.imshow(train_df['seg'][0])
# a

#### Resize images for VGG16

In [ ]:
from skimage.transform import resize
from skimage import io
from PIL import Image

def get_labelled_seg(images_seg):
    image_size = 224
    """
     For segmentation task, read color image, transform it to 3-d one-hot encodeing class target image.
    :param img_path: the image path
    :return: target: (image_size,image_size,21)
    """
    
    train_df_seg224 = np.array([resize(img, (image_size, image_size), Image.ANTIALIAS) for img in images_seg]).astype('int32')    
    train_df_seg_labelled = []
    for img in train_df_seg224:
        print(img.shape)
        idx = (img[:, :, 0] * 256 + img[:, :, 1]) * 256 + img[:, :, 2]
        train_df_seg_labelled.append(np.array(colormap2class[idx], dtype='int64')) # 2-d class map, class of every pixel
        
    return label_map

image_size = 224
train_df_img224 = np.array([resize(img, (image_size, image_size, 3)) for img in train_df["img"]]).astype('float32')
train_df_seg224 = np.array([resize(img, (image_size, image_size)) for img in train_df["seg"]]).astype('float32')

In [ ]:
# Comparing Resizing
lim = 1
for img in train_df_seg224[0:lim]:
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.imshow(img)
    
for img in train_df["seg"][0:lim]:
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.imshow(img)

### 3.2.2 Implementing transfering Learning

We will be implementing the following architecture, U-net follows from this U shape. In the first part the encoder is used to capture the context of the image. In the second part, in the decoder, we will use it to enable the precise localization. AS it can eb seen from the picture below there is no Dense layers above meaning we can take any picture size. Common structure for VGG16 is 224 img size  

![](https://raw.githubusercontent.com/danilotpnta/ComputerVission/main/img/U-net.png)
Img source: https://youtu.be/azM57JuQpQI

The above network uses skip connections, which means we will be skipping some layers in the NN. This skipping helps in traverse the information faster in the NN. It also helps in the problem of vanishing gradient descent. Thus, our Unet network will also account for this.

#### Lets create UNet, and the model

We will load the model imagenet weights and set the layers not to be trainable. The VGG16 represents only the encoder par thus, we need to define the decoder block.

In [ ]:
from keras.utils.vis_utils import plot_model
# Based on tinyurl.com/2ohv3d8z

def upSampling(filter,input,concat,concat_encoder):    
    x = UpSampling2D(size=(2,2),interpolation='bilinear')(input) 
    x = Conv2D(filters=filter,kernel_size=(2,2),activation='relu',padding='same',kernel_initializer='he_normal')(x) 
    up_conv = BatchNormalization()(x)
    
    if concat:
        merged = tf.keras.layers.concatenate([concat_encoder,up_conv])
    else:
        merged = up_conv
    conv1 = BatchNormalization()(Conv2D(filters=filter,kernel_size=(3,3),activation='relu',padding='same',
                                        kernel_initializer='he_normal')(merged))
    conv2 = BatchNormalization()(Conv2D(filters=filter, kernel_size=(3, 3), activation='relu', padding='same',
                                        kernel_initializer='he_normal')(conv1))

    return conv2


def build_unet():
    # Create a VGG16 model, and remove the last layer. This will be replaced with the images classes we have. 
    vgg16 = VGG16(weights='imagenet',include_top = False,input_shape=(image_size,image_size,3))
    
    # Sets layers not trainable
    for layer in vgg16.layers:
        layer.trainable = False
        
    image_feature = vgg16.output

    n_classes = len(labels) + 1 # +1 background
    conv1 = Conv2D(filters=1024,kernel_size=(3,3),activation='relu',padding='same',kernel_initializer='he_normal')(image_feature)
    conv2 = Conv2D(filters=1024, kernel_size=(3,3), activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    decoder_input = Dropout(0.5)(conv2)
    
    s1 = vgg16.get_layer("block1_conv2").output         ## (512 x 512)
    s2 = vgg16.get_layer("block2_conv2").output         ## (256 x 256)
    s3 = vgg16.get_layer("block3_conv2").output         ## (128 x 128)
    s4 = vgg16.get_layer("block4_conv3").output         ## (64 x 64)
    b1 = vgg16.get_layer("block5_conv3").output         ## (32 x 32)

    d1 = upSampling(filter=512,input=decoder_input,concat_encoder=b1,concat=True)
    d2 = upSampling(filter=256,input=d1, concat_encoder=s4,concat=True)
    d3 = upSampling(filter=128,input=d2, concat_encoder=s3,concat=True)
    d4 = upSampling(filter=64, input=d3, concat_encoder=s2,concat=True)
    d5 = upSampling(filter=64, input=d4, concat_encoder=s1,concat=False)

    outputs = Conv2D(filters=n_classes,kernel_size=(1,1),activation='relu',padding='same',
                    kernel_initializer='he_normal',name='seg_output')(d5)

    model = Model(inputs=vgg16.input, outputs=outputs)
    return model


#Creating model object 
# model1 = build_unet()
# model1.summary()
# plot_model(model1, show_shapes=True, show_layer_names=True)

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from keras.utils.vis_utils import plot_model
# Based on: tinyurl.com/2en5hr8k
 
def conv_block(input, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(input)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
 
    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
 
    return x
 
def decoder_block(input, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(input)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x
 
def build_vgg16_unet():
 
    """ Pre-trained VGG16 Model """
    vgg16 = VGG16(weights='imagenet',include_top = False,input_shape= (image_size, image_size, 3))
    
    """ Sets layers not trainable """
    for layer in vgg16.layers:
        layer.trainable = False
 
    """ Encoder """
    s1 = vgg16.get_layer("block1_conv2").output         ## (512 x 512)
    s2 = vgg16.get_layer("block2_conv2").output         ## (256 x 256)
    s3 = vgg16.get_layer("block3_conv3").output         ## (128 x 128)
    s4 = vgg16.get_layer("block4_conv3").output         ## (64 x 64)
 
    """ Bridge """
    b1 = vgg16.get_layer("block5_conv3").output         ## (32 x 32)
 
    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                     ## (64 x 64)
    d2 = decoder_block(d1, s3, 256)                     ## (128 x 128)
    d3 = decoder_block(d2, s2, 128)                     ## (256 x 256)
    d4 = decoder_block(d3, s1, 64)                      ## (512 x 512)
 
    """ Output """
    n_classes = len(labels) + 1 # +1 background
    outputs = Conv2D(n_classes, 1, padding="same", activation="relu")(d4)
 
    model = Model(vgg16.input, outputs, name="VGG16_U-Net")
    return model

# model = build_vgg16_unet()
# model.summary()
# plot_model(model, show_shapes=True, show_layer_names=True)

# Checks if all layers are set to not trainable 
# layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
# model_df = pd.DataFrame(layers, columns=["Layer Type", "Layer Name", "Layer Trainable"])
# display(model_df)

#### Creating the Los function for optimizer

In [ ]:
import tensorflow.keras.backend as K
# Based on: tinyurl.com/2zyh26mg

def dice_coef(y_true, y_pred):
    smooth = 1
    y_pred = K.reshape(y_pred, (-1, K.int_shape(y_pred)[-1]))
    y_true = K.one_hot(tf.compat.v1.to_int32(K.flatten(y_true)), K.int_shape(y_pred)[-1])
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f*y_true_f) + K.sum(y_pred_f*y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1.-dice_coef(y_true, y_pred)

#### Splitting the data

In [ ]:
np.random.seed(12)
from sklearn.model_selection import train_test_split
Xtrain_seg, Xtest_seg, Ytrain_seg, Ytest_seg = train_test_split(train_df_img224,train_df_seg224,test_size=0.1, random_state=12)
print("X_training Images Size:", Xtrain_seg.shape)
print("Y_training Labels Size:", Ytrain_seg.shape)
print("X_test Images Size:", Xtest_seg.shape)
print("Y_test Labels Size:", Ytest_seg.shape)

In [ ]:
# Looking at split data
lim = 1
for img in Xtest_seg[0:lim]:
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.imshow(img)
    
for img in Ytest_seg[0:lim]:
    fig, ax = plt.subplots(figsize=(5, 5))
    plt.imshow(img)

In [ ]:
training_samples_count = Xtrain_seg.shape[0]
lr = 0.0001
epochs = 25
batch_size = 24 
image_size = 224
input_shape = (image_size, image_size, 3)

model1 = build_unet()
model1.compile(optimizer=keras.optimizers.Adam(learning_rate = lr),loss=dice_coef_loss, metrics=[dice_coef])
model1_history = model1.fit(Xtrain_seg, Ytrain_seg, 
                                         batch_size =  batch_size, 
                                         epochs = epochs, 
                                         validation_split=(0.1), shuffle=True)

# model2 = build_vgg16_unet()
# model2.compile(optimizer=keras.optimizers.Adam(learning_rate = lr, decay = lr/epochs ),loss=dice_coef_loss, metrics=[dice_coef])
# model2_history = model.fit(Xtrain_seg, Ytrain_seg, 
#                           batch_size =  batch_size, 
#                           epochs = epochs, 
#                           validation_split=(0.15), shuffle=True)

In [ ]:
# Saving the model weights after training
from tensorflow.keras.models import load_model
model1.save('VGG16_model1.h5')



In [ ]:
""" Performance of the model 1 """
# for key in ['loss', 'val_loss',  'dice_coef', 'val_dice_coef' ]:
#     plt.plot(model1_history.history[key],label=key)
# plt.legend(['Loss', 'Loss: Val ',  'Loss: Dice_coef', 'Loss: Val Dice_coef' ])
# plt.title('Transfer Learning Model Results')
# plt.xlabel("Epochs")
# plt.ylabel("Error")
# plt.savefig('Loss_model_tf')
# plt.show()

![results](https://raw.githubusercontent.com/danilotpnta/ComputerVission/main/img/Loss_model_tf.png)

#### Predicting on the test set

In [ ]:
# Xtrain_seg, Xtest_seg, Ytrain_seg, Ytest_seg = train_test_split(train_df_img224,train_df_seg224,test_size=0.2, random_state=12)
y_pred_model1 = model1.predict(Xtest_seg)
# y_pred_model = model2.predict(Xtest_seg)

In [ ]:
# Model 
y_pred_max = np.argmax(y_pred_model1, axis=3)
print(y_pred_max.shape)

# for i in range(5):
#     plot_img_seg(Xtest_seg[i], Ytest_seg[i], y_pred_max[i])

### Loading model from link

In [ ]:
!wget -O transfer_learning.h5 https://dl.dropboxusercontent.com/s/tm64mq9e0i44lhx/VGG16_model1.h5?dl=0

In [ ]:
testing_tf_model1 = build_unet()
testing_tf_model1.load_weights("transfer_learning.h5")

y_pred_tf_model1 = model1.predict(Xtest_seg)
y_pred_tf_max = np.argmax(y_pred_tf_model1, axis=3)
print(y_pred_tf_max.shape)

# for i in range(5):
#     plot_img_seg(Xtest_seg[i], Ytest_seg[i], y_pred_tf_max[i])

## Submit to competition
You don't need to edit this section. Just use it at the right position in the notebook. See the definition of this function in Sect. 1.3 for more details.

In [ ]:
generate_submission(test_df)

# 4. Adversarial attack
For this part, your goal is to fool your classification and/or segmentation CNN, using an *adversarial attack*. More specifically, the goal is build a CNN to perturb test images in a way that (i) they look unperturbed to humans; but (ii) the CNN classifies/segments these images in line with the perturbations.

# 5. Discussion
Finally, take some time to reflect on what you have learned during this assignment. Reflect and produce an overall discussion with links to the lectures and "real world" computer vision.